In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import Matrix, init_printing

init_printing()

Declarando as valores que nos foram dados:


In [ ]:
#declarando respectivamente tudo que está acima
T_inf = 293.15
L = 0.025
k = 35
U = 67967200
rho = 8618
h_barra = 0.25 * (10**5)
c = 460
euler = 2.718281828459045235360287  # nao foi definido no problema mas estou fazendo por
                                # conta própria por conveniencia caso precise

Definindo a malha, número de elementos, L global , nós, número de equações e etc...


In [ ]:
coord = [
        [0,     0],
        [0.005, 0],
        [0.01,  0],
        [0.015, 0],
        [0.02,  0],
        [0.025, 0],

        [0,     0.05],
        [0.005, 0.05],
        [0.01,  0.05],
        [0.015, 0.05],
        [0.02,  0.05],
        [0.025, 0.05],
]

LG = np.array([
     [1 ,  2 ,  3 ,   4 ,  5 ],
     [2 ,  3 ,  4 ,   5 ,  6 ],
     [8 ,  9 , 10 ,  11 , 12 ],
     [7 ,  8 ,  9 ,  10 , 11 ]
     ])

EQ = [1, 2, 3, 4, 5, 0, 6, 7, 8, 9, 10, 0]

Neq = max(EQ)  # número de equações
Nel = 5  # número de elementos
Nnos = len(EQ) # número de nós


In [ ]:
# Inicializando as matrizes que vamos usar.
K = np.zeros((Neq, Neq))
F = np.zeros(Neq)

Daqui para frente o nosso problema está resumido em montar a matriz K e F local e fazer suas respectivas ligações com a matriz global. 

Mas primeiramente precisamos definir nossas funções de interpolação já vistas em sala de aula e suas respectivas derivadas , tanto em relação a 𝜂 quanto em relação a xi .

In [ ]:

def Phi_1(xi, eta): return ((1 - xi)*(1 - eta)) / 4
def Phi_2(xi, eta): return ((1 + xi)*(1 - eta)) / 4
def Phi_3(xi, eta): return ((1 + xi)*(1 + eta)) / 4
def Phi_4(xi, eta): return ((1 - xi)*(1 + eta)) / 4

def dPhi1_dxi(xi, eta): return (-(1 - eta)) / 4
def dPhi1_deta(xi, eta): return (-(1 - xi)) / 4

def dPhi2_dxi(xi, eta): return (1 - eta) / 4
def dPhi2_deta(xi, eta): return (-(1 + xi)) / 4

def dPhi3_dxi(xi, eta): return (1 + eta) / 4
def dPhi3_deta(xi, eta): return (1 + xi) / 4

def dPhi4_dxi(xi, eta): return (-(1 + eta)) / 4
def dPhi4_deta(xi, eta): return (1 - xi) / 4

E também temos que definir nosso jacobiano:

In [ ]:
# sabendo que jacobiano --> J = D*y
def jacobiano(xi, eta, coords):
    D = np.zeros((2,4))
    D = [
        [dPhi1_dxi(xi, eta) , dPhi2_dxi(xi, eta) , dPhi3_dxi(xi, eta) , dPhi4_dxi(xi, eta) ],
        [dPhi1_deta(xi, eta), dPhi2_deta(xi, eta), dPhi3_deta(xi, eta), dPhi4_deta(xi, eta)]
        ]
    J = np.dot(D,coords)
    return D, J

Vamos usar agora nosso conhecimento da relação da matriz global com a local para achar as coordenadas locais do nosso elemento finito.

In [ ]:
def coord_elemento(e):
    coord_local = np.zeros((4, 2))
    for i in range(4):
        coord_local[i, :] = coord[LG[i][e]-1];
    return coord_local

Agora abaixo, iremos criar algumas ferramentas úteis para nós como Q e os items a e B que nos foram das no enunciado do problema mas agora em formato de código python, e feito isso, iremos montar nossa matriz $ K^e 
 $.

In [ ]:
Q = np.matrix([[1, 0], [0, 1]])

B = (h_barra*L)/k 
a = k/(rho*c)

Q =  Q*a

def K_local(e):
    PG = [-0.5773502691, 0.5773502691]  # pontos de Gauss
    w = [1, 1]                          # pesos
    N_int = len(PG)                     # numero de pontos em cada direcao
    coord_local = coord_elemento(e)

    K_e = np.zeros((4, 4), dtype = object)
    for a in range(N_int): 
        for b in range(N_int):
            D, j = jacobiano(PG[a], PG[b], coord_local)
            det_J = np.linalg.det(j)
            if det_J <= 0:
                print("Determinante negativo!")
                exit()

            B = np.dot(np.linalg.pinv(j),D)
            x = np.dot(B.transpose(),Q)
            x = np.dot(x,B)
            K_e += x * det_J * w[a] * w[b]  # integral de (∇φᵀ)Q(∇φ) dΩ

    return K_e

Mas, com o que foi feito acima podemos já montar a K global também!

In [ ]:
def K_global(K, K_e, e):
    for a in range(4):
        for b in range(4):
            i = EQ[LG[a][e]-1]
            j = EQ[LG[b][e]-1]
            if (i != 0) and (j != 0):   # iremos pular 
                K[i-1][j-1] += K_e[a][b]
    return K

Agora partiremos para o cálculo da F local e de maneira análoga vamos achar a F global também!

In [ ]:
from mpmath.ctx_mp_python import from_man_exp
# tratando condição de contorno
P = np.zeros(Nnos)
P[5] = T_inf
P[11] = T_inf

p = np.zeros(4);

# funcao que vamos usar depois, em formato de funcao lamba 

f = lambda x : U / (rho*c)

def F_prescrito(K_e, e):
  p = P[LG[0:4, e]-1]
  F_p = np.dot(K_e, p)
  return F_p

def F_local(K_e, e):
    PG = [-0.5773502691, 0.5773502691]  # pontos de Gauss
    w = [1, 1]                          # pesos
    N_int = len(PG)                     # numero de pontos em cada direcao
    coord_local = coord_elemento(e)
    M = np.zeros((4,4), dtype=object)
    for a in range (N_int):
        for b in range(N_int):
            do_nothing , J = jacobiano(PG[a], PG[b], coord_local)
            det_J = np.linalg.det(J)
            if det_J <= 0:
                print("Determinante negativo!")
                exit()
            
            Phi = [Phi_1(PG[a], PG[b]), Phi_2(PG[a], PG[b]), Phi_3(PG[a], PG[b]), Phi_4(PG[a], PG[b])]
            W = w[a]*w[b]*det_J
            for m in range(4):
                for n in range(4):
                    M[m][n] += Phi[m]*Phi[n]*W

    f_e = np.zeros(4)
    for a in range(4):
      for j in range(2):
        f_e[a] = f(coord_local[a][j])

    f_e = np.dot(M,f_e)

    return f_e - F_prescrito(K_e, e)

    
def F_global(F, F_e, e):
    for a in range(4):
        i = EQ[LG[a][e]-1]
        if i != 0:
            F[i-1] = F[i-1] + F_e[a]
        
    return F

Agora com todar ferramentas prontas, nós vamos fazer um elemento finito para cada número de elementos que já descobrimos lá encima e chamamos carinhosamente de $N_{el}$.

In [ ]:
for e in range(Nel):
  K_e = K_local(e)
  K_global(K, K_e, e)
  F_e = F_local(K_e, e)
  F_global(F, F_e, e)

E agora comparamos as soluções:

In [ ]:
print("K")
print(K)
print("F")
print(F)

K
[[ 2.97237666e-05 -2.92823245e-05  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.44204412e-05 -1.48618833e-05  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.92823245e-05  5.94475331e-05 -2.92823245e-05  0.00000000e+00
   0.00000000e+00 -1.48618833e-05  2.88408824e-05 -1.48618833e-05
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.92823245e-05  5.94475331e-05 -2.92823245e-05
   0.00000000e+00  0.00000000e+00 -1.48618833e-05  2.88408824e-05
  -1.48618833e-05  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.92823245e-05  5.94475331e-05
  -2.92823245e-05  0.00000000e+00  0.00000000e+00 -1.48618833e-05
   2.88408824e-05 -1.48618833e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.92823245e-05
   5.94475331e-05  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.48618833e-05  2.88408824e-05]
 [ 1.44204412e-05 -1.48618833e-05  0.00000000e+00  0.00000000e+00
   0.00000000e+00  2.97237666e-05 -2.92823245e-05  0.00000000e+00
   0.00000000e+00  0.00000000e+

In [ ]:
solucao_numerica = np.linalg.solve(K,F)

In [ ]:
print(solucao_numerica[0:4])

[900.    875.726 802.904 681.534]


In [ ]:
solucao_analitica = []
xs = np.arange(0, 0.02, 0.005)
t = lambda x : T_inf + (U*L**2)/(2*k)*(1 - (x/L)**2)
for x in xs:
  solucao_analitica.append(t(x))

print(solucao_analitica)

[900.0000000000001, 875.7260000000001, 802.9040000000002, 681.5340000000001]


In [ ]:
print("\nA solução analítica é:\n")
print(solucao_analitica)
print("\nA solução numérica é:\n")
print(solucao_numerica)


A solução analítica é:

[900.0000000000001, 875.7260000000001, 802.9040000000002, 681.5340000000001]

A solução numérica é:

[900.    875.726 802.904 681.534 511.616 900.    875.726 802.904 681.534
 511.616]
